In [ ]:
import sys
import os
import geopandas as gpd
import pandas as pd
import numpy as np

sys.path.append(os.path.abspath(os.path.join(os.pardir)))

from VASA import VASA

In [ ]:
counties_map = gpd.GeoDataFrame \
    .from_features(gpd.read_file("../data/ca_cbg")) \
    .set_crs(epsg=4326) \
    .to_crs("+proj=aea +lat_1=20 +lat_2=60 +lat_0=40 +lon_0=-96 +x_0=0 +y_0=0 +datum=NAD83 +units=m +no_defs")

In [ ]:
fire2019 = pd.read_csv("../data/lakefire2019_data.csv")
fire2019.origin_census_block_group = "0" + fire2019.origin_census_block_group.astype(str)

In [ ]:
unique_cbgs = np.unique(fire2019.origin_census_block_group)
unique_tracts = np.unique([g[:8] for g in unique_cbgs])
unique_counties = np.unique([g[:5] for g in unique_tracts])

In [ ]:
tract_map = counties_map[[(cbg[:8] in unique_tracts) for cbg in counties_map.GEOID]]

In [ ]:
lake_fire = gpd.read_file("../data/lake_fire") \
    .set_crs(epsg=4326) \
    .to_crs("+proj=aea +lat_1=20 +lat_2=60 +lat_0=40 +lon_0=-96 +x_0=0 +y_0=0 +datum=NAD83 +units=m +no_defs")

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(5, 5))

base_map = counties_map[[(cbg[:5] in unique_counties) for cbg in counties_map.GEOID]].copy()

base_map["c"] = "gray"
base_map.loc[[(cbg[:8] in unique_tracts) for cbg in base_map.GEOID], "c"] = "blue"
base_map.loc[[(cbg in unique_cbgs) for cbg in base_map.GEOID], "c"] = "green"

base_map.plot(ax=ax, color=base_map["c"])
lake_fire.plot(ax=ax, color="red")

In [ ]:
v = VASA(
    df=fire2019[["origin_census_block_group", "date_range_start", "distance_traveled_from_home"]],
    gdf=tract_map,
    df_group_col="origin_census_block_group",
    gdf_group_col="GEOID",
    date_col="date_range_start",
    temp_res="week",
    date_format="%Y-%m-%d %H:%M:%S" # %z
)

In [ ]:
del fire2019

In [ ]:
v.fill_missing()

In [ ]:
v.show_weights_connection(k=15)

In [12]:
v.show_weights_connection(band=10_000)

In [ ]:
v.show_weights_connection(k=10, type="union")

In [ ]:
v.show_weights_connection(type="queens")